In [6]:
%pip install azure-storage-blob
%pip install psycopg2 sqlalchemy
%pip install sqlalchemy psycopg2-binary
%pip install google-cloud-storage
%pip install google-cloud-bigquery



Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
import psycopg2
from google.cloud import storage

In [ ]:
# Extracting data from source and storing in Google Cloud

# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Azure Storage connection string and container name
CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
container_name = "otcgroup6"
blob_name = "otc_data.csv"

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Create a blob client using the container name and blob name
#container_client = blob_service_client.get_container_client(CONTAINER_AZURE)
blob_client = blob_service_client.get_blob_client(container=container_name, blob=os.path.basename(file_path))

# Upload the csv file to Azure Blob Storage
with open(file_path, "rb") as data:
    blob_client.upload_blob(data)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {container_name}")


In [ ]:
# Reading data stored in Google Cloud and making a copy of the data frame to do ETL.

# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Azure Storage connection string and container name
CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
container_name = "otcgroup6"

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Create a blob client using the container name and blob name
blob_client = blob_service_client.get_container_client(container_name)

otc_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = blob_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = blob_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))

    # Display the head of the DataFrame
    print(df.shape)

    otc_df = df.copy()


otc_df.head()

In [5]:
otc_df.columns

Index(['SECID', 'CompID', 'Symbol', 'CUSIP', 'Issue', 'Venue', 'SecType',
       'Class', 'CaveatEmptor', 'TierName', 'DAD_PAL', 'ClosingInsideBidPrice',
       'ClosingInsideBidPriceDate', 'ClosingInsideAskPrice',
       'ClosingInsideAskPriceDate', 'InsideBid_AskMidPrice', 'OpenPrice',
       'HighPrice', 'LowPrice', 'LastPrice', 'PreviousClosePrice',
       'PreviousCloseDate', 'ShareVolume', 'DollarVol', 'TradeCount',
       'OTCLinkDolVol', 'OTCLinkShareVol', 'OTCLinkExecCount', 'SHOFlag',
       'Rule3210Flag', 'ShortIntVol', 'ShortInterestDate', 'MMIDCount',
       'BFCMmid', 'TierID', 'ClosingBestBid', 'ClosingBestBidDate',
       'BestBidSource', 'ClosingBestAsk', 'ClosingBestAskDate',
       'BestAskSource', 'SecurityStatus', 'SharesOutstanding',
       'SharesOutstandingAsOfDate'],
      dtype='object')

In [6]:
otc_df.head()

,SECID,CompID,Symbol,CUSIP,Issue,Venue,SecType,Class,CaveatEmptor,TierName,...,TierID,ClosingBestBid,ClosingBestBidDate,BestBidSource,ClosingBestAsk,ClosingBestAskDate,BestAskSource,SecurityStatus,SharesOutstanding,SharesOutstandingAsOfDate
0,100010,649124,GPGPF,G4205Y111,GUINESS PEAT GRP PLC,Grey Market,Ordinary Shares,NaN,N,Grey Market,...,30.0,NaN,2011-09-01,NaN,NaN,2011-09-01,NaN,Active,NaN,NaN
1,100010,649124,GPGPF,G4205Y111,GUINESS PEAT GRP PLC,Grey Market,Ordinary Shares,NaN,N,Grey Market,...,30.0,NaN,2011-09-02,NaN,NaN,2011-09-02,NaN,Active,NaN,NaN
2,100010,649124,GPGPF,G4205Y111,GUINESS PEAT GRP PLC,Grey Market,Ordinary Shares,NaN,N,Grey Market,...,30.0,NaN,2011-09-06,NaN,NaN,2011-09-06,NaN,Active,NaN,NaN
3,100010,649124,GPGPF,G4205Y111,GUINESS PEAT GRP PLC,Grey Market,Ordinary Shares,NaN,N,Grey Market,...,30.0,NaN,2011-09-07,NaN,NaN,2011-09-07,NaN,Active,NaN,NaN
4,100010,649124,GPGPF,G4205Y111,GUINESS PEAT GRP PLC,Grey Market,Ordinary Shares,NaN,N,Grey Market,...,30.0,NaN,2011-09-08,NaN,NaN,2011-09-08,NaN,Active,NaN,NaN


In [7]:
# Transformation:

# remove the following columns because they are irrelevant to our project.
columns_to_remove = ['Class', 'CaveatEmptor', 'DAD_PAL', 'SHOFlag', 'Rule3210Flag', 'BestBidSource', 'BestAskSource'] 
 
# Drop the specified columns from the DataFrame
otc_df = otc_df.drop(columns=columns_to_remove)
 
# Display the modified DataFrame
print(otc_df.head())

    SECID  CompID Symbol      CUSIP                 Issue        Venue  \
0  100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
1  100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
2  100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
3  100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
4  100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   

           SecType     TierName  ClosingInsideBidPrice  \
0  Ordinary Shares  Grey Market                    NaN   
1  Ordinary Shares  Grey Market                    NaN   
2  Ordinary Shares  Grey Market                    NaN   
3  Ordinary Shares  Grey Market                    NaN   
4  Ordinary Shares  Grey Market                    NaN   

  ClosingInsideBidPriceDate  ...  MMIDCount BFCMmid  TierID  ClosingBestBid  \
0                       NaN  ...        NaN     NaN    30.0             NaN   
1                       NaN  ...        NaN     

In [23]:
print(otc_df.columns)


Index(['SECID', 'CompID', 'Symbol', 'CUSIP', 'Issue', 'Venue', 'SecType',
       'TierName', 'ClosingInsideBidPrice', 'ClosingInsideBidPriceDate',
       'ClosingInsideAskPrice', 'ClosingInsideAskPriceDate',
       'InsideBid_AskMidPrice', 'OpenPrice', 'HighPrice', 'LowPrice',
       'LastPrice', 'PreviousClosePrice', 'PreviousCloseDate', 'ShareVolume',
       'DollarVol', 'TradeCount', 'OTCLinkDolVol', 'OTCLinkShareVol',
       'OTCLinkExecCount', 'ShortIntVol', 'ShortInterestDate', 'MMIDCount',
       'BFCMmid', 'TierID', 'ClosingBestBid', 'ClosingBestBidDate',
       'ClosingBestAsk', 'ClosingBestAskDate', 'SecurityStatus',
       'SharesOutstanding', 'SharesOutstandingAsOfDate'],
      dtype='object')


In [24]:
print(otc_df[otc_df[['OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice']].notna().all(axis=1)]
      [['OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice']].head())

      OpenPrice  HighPrice  LowPrice  LastPrice
66         0.50       0.50      0.50       0.50
306        0.45       0.45      0.45       0.45
2282      86.00      86.00     86.00      86.00
2289      90.00      90.00     90.00      90.00
2338      86.00      95.00     86.00      95.00


In [25]:

# Remove rows that contain NaN values in specified columns and create a new DataFrame
#Since we are looking at the OTC market transaction, we do not want prcies which are NaNs.
cleaned_otc_df = otc_df.dropna(subset=['OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice'])

# Display the first few rows of the new cleaned DataFrame
print(cleaned_otc_df.head())

       SECID  CompID Symbol      CUSIP                 Issue        Venue  \
66    100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
306   100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
2282   10002   26620   DAFL  262507106       DU ART FILM LAB     OTC Link   
2289   10002   26620   DAFL  262507106       DU ART FILM LAB     OTC Link   
2338   10002   26620   DAFL  262507106       DU ART FILM LAB     OTC Link   

              SecType                 TierName  ClosingInsideBidPrice  \
66    Ordinary Shares              Grey Market                    NaN   
306   Ordinary Shares              Grey Market                    NaN   
2282     Common Stock  OTC Pink No Information                   86.0   
2289     Common Stock  OTC Pink No Information                   86.0   
2338     Common Stock  OTC Pink No Information                   86.0   

     ClosingInsideBidPriceDate  ...  MMIDCount BFCMmid  TierID  \
66                         NaN  

In [29]:
print(cleaned_otc_df.shape)


(2431388, 37)


In [30]:
print(cleaned_otc_df.shape)


(2431388, 37)


In [28]:
print(cleaned_otc_df.head())

       SECID  CompID Symbol      CUSIP                 Issue        Venue  \
66    100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
306   100010  649124  GPGPF  G4205Y111  GUINESS PEAT GRP PLC  Grey Market   
2282   10002   26620   DAFL  262507106       DU ART FILM LAB     OTC Link   
2289   10002   26620   DAFL  262507106       DU ART FILM LAB     OTC Link   
2338   10002   26620   DAFL  262507106       DU ART FILM LAB     OTC Link   

              SecType                 TierName  ClosingInsideBidPrice  \
66    Ordinary Shares              Grey Market                    NaN   
306   Ordinary Shares              Grey Market                    NaN   
2282     Common Stock  OTC Pink No Information                   86.0   
2289     Common Stock  OTC Pink No Information                   86.0   
2338     Common Stock  OTC Pink No Information                   86.0   

     ClosingInsideBidPriceDate  ...  MMIDCount BFCMmid  TierID  \
66                         NaN  

In [31]:
# Getting a CSV file of the cleaned data.

cleaned_otc_df.to_csv('cleaned_otc_df.csv', index=False)

In [8]:
# Loading the data into Big Query

from google.cloud import bigquery

# Set up your Google Cloud project ID
project_id = 'cis4400-group-project-423420'

# Set the path to your service account key file
service_account_key_file = 'GC_config.json'

# Set up BigQuery client with service account credentials
client = bigquery.Client.from_service_account_json(service_account_key_file, project=project_id)

# Define your dataset ID
dataset_id = 'cis4400GP'

# Create the BigQuery dataset if it doesn't exist
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset = client.create_dataset(dataset, exists_ok=True)  # Use exists_ok=True to avoid errors if the dataset already exists

# Set up BigQuery table information
table_id = 'OTC_data'

# Create the BigQuery table with schema auto-detection (without specifying schema)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig(
    autodetect=True,  # Automatically detect schema from CSV file
    skip_leading_rows=1,  # Skip the header row
    source_format=bigquery.SourceFormat.CSV
)

# Load CSV file into BigQuery table
with open('cleaned_otc_df.csv', 'rb') as source_file:
    job = client.load_table_from_file(
        source_file,
        table_ref,
        job_config=job_config
    )

job.result()  # Waits for the job to complete

print(f"Loaded {job.output_rows} rows into {dataset_id}.{table_id}.")


Loaded 2431388 rows into cis4400GP.OTC_data.


In [22]:
# The following are miscellaneous codes for testing and debugging

# These are test codes we did to check if data needed to be cleaned.
# Get distinct values in a specific field (column_name)
distinct_values = otc_df['Class'].unique()
 
# Display distinct values
print(distinct_values)

[nan 'N' 'A' 'M' 'B' 'C' 'D' 'R' 'Z' 'H' 'P']


In [5]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Azure Storage connection string and container name
CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
container_name = "otcgroup6"
blob_name = "otc_data.csv"

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Create a blob client using the container name and blob name
#container_client = blob_service_client.get_container_client(CONTAINER_AZURE)
blob_client = blob_service_client.get_blob_client(container=container_name, blob=os.path.basename(file_path))

# Upload the csv file to Azure Blob Storage
with open(file_path, "rb") as data:
    blob_client.upload_blob(data)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {container_name}")


Uploaded otc to Azure Blob Storage in container otcgroup6


In [3]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Azure Storage connection string and container name
CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
container_name = "otcgroup6"

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Create a blob client using the container name and blob name
blob_client = blob_service_client.get_container_client(container_name)

otc_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = blob_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = blob_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))

    # Display the head of the DataFrame
    print(df.shape)

    otc_df = df.copy()


otc_df.head()

otc_data.csv


C:\Users\j.rhee\AppData\Local\Temp\ipykernel_19780\2757125731.py:27: DtypeWarning: Columns (7,10,28,29,37,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


(13124597, 44)
